In [1]:
import os
import json
import mlflow

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_aws import BedrockLLM

import sagemaker
import boto3

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/willem/.config/sagemaker/config.yaml


In [2]:
mlflow.set_tracking_uri("http://localhost:5000")

In [3]:
session = boto3.Session(profile_name='vrt-analytics-engineer-nonsensitive')
sagemaker_session = sagemaker.Session(boto_session=session)
role = sagemaker.get_execution_role(sagemaker_session=sagemaker_session)

In [2]:
# 1. Get retriever
from langchain_aws.retrievers import AmazonKnowledgeBasesRetriever

retriever = AmazonKnowledgeBasesRetriever(
    knowledge_base_id="PUIJP4EQUA",
    retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 4}},
)

/home/willem/miniconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/willem/miniconda3/envs/llm/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
# 2. Get LLM
llm = BedrockLLM(
    credentials_profile_name="vrt-analytics-engineer-nonsensitive", 
    region_name="eu-west-2",
    model_id="meta.llama3-70b-instruct-v1:0"
)

In [8]:
# 3. Create model in MLFLOW
mlflow.set_experiment("VRT MAX RAG v1")

retrievalQA = RetrievalQA.from_llm(llm=llm, retriever=retriever)

persist_dir = "../data/faiss"
with mlflow.start_run() as run:
    model_info = mlflow.langchain.log_model(
        retrievalQA,
        artifact_path="rag",
        loader_fn=load_retriever,
        persist_dir=persist_dir,
    )

2024/08/06 14:57:02 INFO mlflow.tracking.fluent: Experiment with name 'VRT MAX RAG v1' does not exist. Creating a new experiment.
2024/08/06 14:57:09 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp57qx3nxm/model, flavor: langchain). Fall back to return ['langchain==0.2.8', 'pydantic==2.8.2', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback. 
